In [4]:
import numpy as np
import pandas as pd
import torch_geometric
import networkx as nx
from torch_geometric.data import Data, Dataset
from torch_geometric.datasets.planetoid import Planetoid
from torch_geometric.transforms.to_undirected import ToUndirected
import torch

In [6]:
!head /home/jrm28/fairness/NeuralCommonNeighbor/dataset/reliant_data/bail/bail_edges.txt

0.000000000000000000e+00 3.597000000000000000e+03
0.000000000000000000e+00 5.975000000000000000e+03
1.000000000000000000e+00 1.900000000000000000e+03
1.000000000000000000e+00 2.561000000000000000e+03
1.000000000000000000e+00 4.685000000000000000e+03
1.000000000000000000e+00 5.510000000000000000e+03
1.000000000000000000e+00 6.027000000000000000e+03
1.000000000000000000e+00 6.375000000000000000e+03
1.000000000000000000e+00 6.602000000000000000e+03
1.000000000000000000e+00 8.050000000000000000e+03


In [5]:
G = np.load('/home/jrm28/fairness/NeuralCommonNeighbor/dataset/reliant_data/bail/split/test_split.npy')

In [3]:
edge_array = np.array([(u, v) for u, v in G.edges])
edge_array.sort()

In [4]:
edge_array

array([[2849, 3021],
       [2849, 3335],
       [2849, 3299],
       ...,
       [2691, 2792],
       [3268, 3407],
       [2788, 3271]])

In [5]:
features = np.loadtxt('/home/jrm28/fairness/subgraph_sketching-original/dataset/ego-facebook/raw/facebook/1684.feat')

# sort features by node id
sorted_idx = features[:, 0].argsort()
features = features[sorted_idx, :]


# Map node to index
node_mapper = np.arange(features.shape[0])
node_mapper = dict(zip(features[:, 0].astype(int), node_mapper))

for edge in edge_array:
    edge[0] = node_mapper[edge[0]]
    edge[1] = node_mapper[edge[1]]
    
edge_array

array([[204, 376],
       [204, 690],
       [204, 654],
       ...,
       [ 46, 147],
       [623, 762],
       [143, 626]])

In [6]:
transform = ToUndirected()

# feature 147 will be the sensitive attribute
x = torch.tensor(features[:, 1:], dtype=torch.float32)
y = x[:, 147]

# Removing columns 147 and 148 (sensitive attributes)
x = torch.cat((x[:, :147], x[:, 149:]), dim=1)

facebook = Data(x=x, y=y, edge_index=torch.tensor(edge_array.T))
facebook = transform(facebook)

In [7]:
torch.save(facebook, '/Users/joaopedromattos/Documents/fairness/subgraph_sketching-original/dataset/facebook/processed/facebook_1684.pt')

## EDA

In [9]:
torch.vstack((y[facebook.edge_index[0]], y[facebook.edge_index[1]])).sum(axis=0)  

tensor([0., 0., 1.,  ..., 0., 1., 0.])

In [23]:
(torch.vstack((y[facebook.edge_index[0]], y[facebook.edge_index[1]])).sum(axis=0) == 0).sum() + (torch.vstack((y[facebook.edge_index[0]], y[facebook.edge_index[1]])).sum(axis=0) == 2).sum(), (torch.vstack((y[facebook.edge_index[0]], y[facebook.edge_index[1]])).sum(axis=0) == 1).sum(), 

(tensor(15978), tensor(12070))